In [ ]:
import zipfile
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# open g_cpc_title.tsv.zip to a dataframe without extracting    
with zipfile.ZipFile('g_cpc_title.tsv.zip', 'r') as zip_ref:
    cpc_title = pd.read_csv(zip_ref.open('g_cpc_title.tsv'), sep='\t')



In [ ]:
# open g_cpc_current.tsv.zip to a dataframe without extracting
with zipfile.ZipFile('g_cpc_current.tsv.zip', 'r') as zip_ref:
    cpc_current = pd.read_csv(zip_ref.open('g_cpc_current.tsv'), sep='\t')

In [ ]:
with zipfile.ZipFile('citations_partmerged.csv.zip', 'r') as zip_ref:
    citations = pd.read_csv(zip_ref.open('citations_partmerged.csv'))

In [ ]:
citations = citations.drop(columns={'Unnamed: 0'})

In [ ]:
cpc_current['patent_id'] = cpc_current['patent_id'].astype(str)

In [ ]:
citations = citations.merge(cpc_current[['patent_id','cpc_subclass']], left_on='citation_patent_id', right_on='patent_id', how='inner')

In [ ]:
# Define the size of each chunk
chunk_size = 500000  # Adjust based on memory limits
chunks = []

# Split 'citations' into chunks
for i in range(0, len(citations), chunk_size):
    chunk = citations.iloc[i:i + chunk_size]
    
    # Perform the merge on each chunk
    merged_chunk = chunk.merge(cpc_current[['patent_id', 'cpc_subclass']], 
                               left_on='citation_patent_id', 
                               right_on='patent_id', 
                               how='inner')
    
    # Drop the extra 'patent_id' column from 'cpc_current' after the merge
    merged_chunk = merged_chunk.drop(columns=['patent_id_y']).rename(columns={'patent_id_x':'patent_id','cpc_subclass':'cited_subclass'})
    
    # Perform groupby operations (example: counting occurrences of each subclass)
    grouped_chunk = merged_chunk.groupby(['citation_patent_id', 'cpc_subclass']).size().reset_index(name='count')
    
    # Append the result to the list
    chunks.append(grouped_chunk)

# Concatenate all chunks back into a single DataFrame
final_result = pd.concat(chunks, ignore_index=True)


In [ ]:
# Define the size of each chunk
chunk_size = 500000  # Adjust based on memory limits

# Define a counter for file naming and progress tracking
file_counter = 1

# Split 'citations' into chunks
for i in range(0, len(citations), chunk_size):
    chunk = citations.iloc[i:i + chunk_size]
    
    # Perform the merge on each chunk
    merged_chunk = chunk.merge(cpc_current[['patent_id', 'cpc_subclass']], 
                               left_on='citation_patent_id', 
                               right_on='patent_id', 
                               how='inner')
    
    # Drop the extra 'patent_id' column from 'cpc_current' after the merge
    merged_chunk = merged_chunk.drop(columns=['patent_id_y']).rename(columns={'patent_id_x':'patent_id','cpc_subclass':'cited_subclass'})
    
    # Perform groupby operations (example: counting occurrences of each subclass)
    grouped_chunk = merged_chunk.groupby(['citing_subclass', 'cited_subclass']).size().reset_index(name='count')
    
    # Save the grouped chunk to a file
    file_name = f'grouped_chunk_{file_counter}.csv'
    grouped_chunk.to_csv(file_name, index=False)
    
    # Print progress
    print(f"Processed chunk {file_counter}, saved as {file_name}")
    
    # Increment the counter for the next file
    file_counter += 1

print("All chunks processed and saved.")


In [ ]:
grouped_chunk

In [ ]:
pd.read_csv('grouped_chunk_1.csv')

In [ ]:
merged = pd.DataFrame()
for i in range(1,590):
    file_name = f'grouped_chunk_{i}.csv'
    chunk = pd.read_csv(file_name)
    merged = pd.concat([merged,chunk], ignore_index=True)
    merged = merged.groupby(['citing_subclass','cited_subclass'], as_index=False)['count'].sum()
    print(f'file {i} merged')

merged.to_csv('cpc_cross_citation.csv', index=False)
    


In [ ]:
import numpy as np

In [ ]:
def gen(x):
    return 1-sum(x**2)/sum(x)**2

In [ ]:
hhi(np.array([1,2,3]))

In [ ]:
subclass_generality = merged.groupby('cited_subclass')['count'].apply(gen)

In [ ]:
subclass_generality = subclass_generality.reset_index().rename(columns={'cited_subclass':'subclass','count':'generality'})

In [ ]:
subclass_generality = subclass_generality.sort_values('generality', ascending=False)

In [ ]:
subclass_generality[:20]

In [ ]:
plt.plot(np.log(subclass_generality['generality']))
plt.show()

In [ ]:
test = test.merge(cpc_current[['patent_id', 'cpc_subclass']], 
                               left_on='citation_patent_id', 
                               right_on='patent_id', 
                               how='inner').drop(columns=['patent_id_y']).rename(columns={'patent_id_x':'patent_id','cpc_subclass':'cited_subclass'})

In [ ]:
test.groupby(['citing_subclass','cited_subclass']).size().reset_index(name='count')